# Accessing webservices

How do you get the URLs? You can query a Catalogue Service Web (CSW) endpoint, but the easiest way is just to pull it from the AuScope portal.

In [4]:
from utilities import website, embed_map

website('http://portal.auscope.org', 'AuScope Portal', width=1000, height=500)

Once you've got your endpoint, you can hit it up for some data.

In [5]:
url = 'http://geossdi.dmp.wa.gov.au:80/services/wfs'

Here we use the requests library to hit the endpoint, but you can use anything you like really...

In [7]:
import requests

# Get data from WA, get me the first 5 gsml:Borehole items
params = dict(
    service='WFS',
    version='2.0.0',
    request='GetFeature',
    typename='gsml:Borehole',
    bbox='-36.669908, 110.630435, -12.685026, 129.614808',
    max_features=5
)

# Construct the request & give me the response
response = requests.get(url, params=params)
print (response.url)

http://geossdi.dmp.wa.gov.au:80/services/wfs?service=WFS&version=2.0.0&request=GetFeature&typename=gsml%3ABorehole&bbox=-36.669908%2C+110.630435%2C+-12.685026%2C+129.614808&max_features=5


What's the response?

In [8]:
print (response.content)

b'<?xml version="1.0" encoding="UTF-8"?><wfs:FeatureCollection xmlns:wfs="http://www.opengis.net/wfs/2.0" xmlns:xs="http://www.w3.org/2001/XMLSchema" xmlns:gsml="urn:cgi:xmlns:CGI:GeoSciML:2.0" xmlns:mo="http://xmlns.geoscience.gov.au/minoccml/1.0" xmlns:gsmlp="http://xmlns.geosciml.org/geosciml-portrayal/4.0" xmlns:mt="http://xmlns.geoscience.gov.au/mineraltenementml/1.0" xmlns:sa="http://www.opengis.net/sampling/1.0" xmlns:nvcl="http://www.auscope.org/nvcl" xmlns:om="http://www.opengis.net/om/1.0" xmlns:er="urn:cgi:xmlns:GGIC:EarthResource:1.1" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" numberMatched="unknown" numberReturned="200" timeStamp="2018-07-30T01:22:26.766Z" xsi:schemaLocation="urn:cgi:xmlns:CGI:GeoSciML:2.0 http://www.geosciml.org/geosciml/2.0/xsd/geosciml.xsd http://www.opengis.net/gml/3.2 http://schemas.opengis.net/gml/3.2.1/gml.xsd http://www.opengis.net/wfs/2.0 http://schemas.opengis.net/wfs/2.0/wfs.xsd"><wfs:member><gsml:Borehole id="gsml.borehole.07THD002">

Most OGC webservices support RESTful queries through the URL like this, but for more complex queries with multiple filters (i.e. not just the bounding box) you might want to use the SOAPy method of posting an XML query to the server. 

    <?xml version="1.0" encoding="utf-8"?>
    <wfs:GetFeature
     xmlns:wfs="http://www.opengis.net/wfs"
     xmlns:ogc="http://www.opengis.net/ogc"
     xmlns:gml="http://www.opengis.net/gml"
     xmlns:sa="http://www.opengis.net/sampling/1.0"
     maxFeatures="{max_features}" service="WFS"
     version="1.1.0" xmlns:gsml="urn:cgi:xmlns:CGI:GeoSciML:2.0">
       <wfs:Query typeName="gsml:Borehole">
        <ogc:Filter xmlns:ogc="http://www.opengis.net/ogc">
           <ogc:BBOX>
             <ogc:PropertyName>gsml:shape</ogc:PropertyName>
             <gml:Envelope srsName="EPSG:4326">
               <gml:lowerCorner>
                 {lower_corner[0]}, {lower_corner[1]}
               </gml:lowerCorner>
               <gml:upperCorner>
                 {upper_corner[0]}, {upper_corner[1]}
               </gml:upperCorner>
             </gml:Envelope>
           </ogc:BBOX>
         </ogc:Filter>
       </wfs:Query>
    </wfs:GetFeature>

## Using OWSLib

OWSLib is a library which makes forming OGC webservice queries much easier

In [9]:
# Products to pull
aster_products = [
    'AlOH_group_composition',
    'Ferric_oxide_composition',
    'MgOH_group_composition'
]
bounds = (119.52, -21.6, 120.90, -20.5)
wcsurl = 'http://aster.nci.org.au/thredds/wcs/aster/vnir/Aus_Mainland/Aus_Mainland_{0}_reprojected.nc4'

In [10]:
from owslib import wcs
import socket

# Only uncomment this if you're not giving a talk and can afford to wait for the ~18 Mb download
# Otherwise, pull the geoTIFFs out of the geotiffs folder.
for product in aster_products:
    try:
        coverage_service = wcs.WebCoverageService(wcsurl.format(product))
        response = coverage_service.getCoverage(product,
                                                bbox=bounds,       # We only want data within the bounds we're considering
                                                format='GeoTIFF',  # We want data in GeoTIFF format
                                                timeout=100)       # We need to wait a bit longer for large files...
        with open(product + '.gtif', 'wb') as fhandle:
            fhandle.write(response.read())
    except socket.timeout:
        continue

ConnectionError: HTTPConnectionPool(host='aster.nci.org.au', port=80): Max retries exceeded with url: /thredds/wcs/aster/vnir/Aus_Mainland/Aus_Mainland_AlOH_group_composition_reprojected.nc4?service=WCS&request=GetCapabilities (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x00000237B6ABECC0>: Failed to establish a new connection: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond',))

OWSLib knows how to talk to WCS, WFS, WMS etc etc etc